# Đánh giá khớp (Hồi quy đa thức)

Trong notebook này, chúng ta sẽ so sánh các mô hình hồi quy khác nhau để đánh giá xem mô hình nào khớp nhất. Chúng ta sẽ sử dụng hồi quy đa thức để xem xét chủ đề này. Cụ thể, chúng ta sẽ:
* Viết hàm nhận một đặc trưng đơn lẻ (trong mảng Numpy/Series) và một bậc, trả về một DataFrame có từng cột là đặc trưng tới giá trị đa thức lên tới tổng bậc, chẳng hạn: bậc = 3 thì cột 1 là đặc trưng ban đầu, cột 2 là đặc trưng bình phương và cột 3 là đặc trưng lập phương.
* Sử dụng matplotlib để trực quan hóa hồi quy đa thức.
* Sử dụng matplotlib để trực quan hóa cùng bậc đa thức trong cùng một tập con dữ liệu.
* Sử dụng tập kiểm định để chọn bậc đa thức.
* Đánh giá khớp cuối cùng sử dụng dữ liệu kiểm tra.

Chúng ta vẫn sử dụng dữ liệu nhà ở từ notebook trước.

## Thư viện

In [1]:
import sklearn
import pandas
import numpy as np

C:\Users\Bang\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Tiếp theo, chúng ta sẽ viết hàm đa thức nhận một đặc trưng và một bậc cực đại, trả về DataFrame chứa tất cả các lũy thừa của đặc trưng cho đến bậc cực đại.

Cách đơn giản nhất để vận dụng lũy thừa vào một Series là sử dụng các hàm `.apply` và `lambda x:`. Tuy nhiên, mảng Numpy chỉ có thể thực hiện nó dễ dàng với các phép số học đơn giản; nếu không, nó sẽ yêu cầu `np.vectorize` khó hiểu hơn. Ví dụ để lấy mảng mock và tính lũy thừa bậc ba, chúng ta có thể làm như sau:

In [2]:
mock = np.array([1., 2., 3.])
mock_series = pandas.Series(mock)
mock_cubed = mock ** 3
mock_cubed_long = np.vectorize(lambda x: x ** 3)(mock)
mock_series_cubed = mock_series.apply(lambda x: x ** 3)
print(mock, mock_cubed, mock_cubed_long)
print(mock_series, "\n", mock_series_cubed)

[1. 2. 3.] [ 1.  8. 27.] [ 1.  8. 27.]
0    1.0
1    2.0
2    3.0
dtype: float64 
 0     1.0
1     8.0
2    27.0
dtype: float64


Chúng ta có thể tạo một DataFrame trống sử dụng `pandas.DataFrame()`, sau đó thêm bất kỳ cột nào vào đó với `dataframe['column_name'] = value`. Chẳng hạn, chúng ta tạo một DataFrame trống và đặt cột 'power_1' là lũy thừa bậc một của giá trị mock.

In [3]:
data = pandas.DataFrame()
data['power_1'] = mock
print(data)

   power_1
0      1.0
1      2.0
2      3.0


## Hàm `polynomial_dataframe`

Sử dụng các gợi ý phía trên để hoàn thiện hàm sau:

In [4]:
def polynomial_dataframe(feature, degree):
    # giả sử bậc >= 1
    # khởi tạo DataFrame:
    poly_data = pandas.DataFrame()
    # và đặt poly_data['power_1'] bằng với đặc trưng đã truyền vào
    poly_data["power_1"] = feature
    # trước tiên kiểm tra xem bậc có lớn hơn 1 không
    if degree > 1:
        # thì lặp qua các bậc còn lại:
        # range thường bắt đầu range từ 0 và kết thúc ở đầu cuối -1. Chúng ta cần nó bắt đầu từ 2 và ngừng ở bậc
        for power in range(2, degree+1):
            # trước tiên đặt tên cột:
            name = 'power_' + str(power)
            # sau đó gán poly_sframe[name] cho lũy thừa thích hợp của đặc trưng
            poly_data[name] = np.vectorize(lambda x:'{:.2e}'.format(x**power))(feature)
    return poly_data

Kiểm tra hàm với biến **mock** và kết quả dự kiến của cell sau:

In [5]:
print(polynomial_dataframe(mock, 3))

   power_1   power_2   power_3
0      1.0  1.00e+00  1.00e+00
1      2.0  4.00e+00  8.00e+00
2      3.0  9.00e+00  2.70e+01


# Trực quan hóa hồi quy đa thức

Sử dụng matplotlib để hiển thị hồi quy đa thức trông như thế nào trong dữ liệu thực.

In [6]:
full_data = pandas.read_csv("kc_house_data.csv", index_col=0)

Chúng ta sẽ dùng biến sqft_living. Để vẽ biểu đồ (nối các chấm), chúng ta cần sắp xếp giá trị của sqft_living. Những ngôi nhà có diện tích giống nhau sẽ phân biệt theo giá.

In [7]:
full_data = full_data.sort_values(['sqft_living', 'price'])

Hãy bắt đầu với đa thức bậc 1 sử dụng 'sqft_living' (chẳng hạn 1 dòng) để dự đoán 'price' và vẽ biểu đồ xem nó trông như thế nào.

In [8]:
poly1_data = polynomial_dataframe(full_data['sqft_living'], 1)
poly1_data['price'] = full_data['price'] # add price to the data since it's the target

Đó là dữ liệu. Đã đến lúc tạo mô hình và thử với nó.

In [9]:
from sklearn.linear_model import LinearRegression
features_poly1 = np.reshape(poly1_data['power_1'].values, [-1, 1])
labels_poly1 = poly1_data['price'].values
model1 = LinearRegression().fit(features_poly1, labels_poly1)

In [10]:
#hãy xem các trọng số trước khi vẽ biểu đồ
model1.coef_

array([280.6235679])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(poly1_data['power_1'],poly1_data['price'],'.',
        poly1_data['power_1'], model1.predict(features_poly1),'-')

Hãy giải nén lệnh plt.plot(). Cặp đầu tiên là lũy thừa bậc 1 của sqft và giá thực, sau đó yêu cầu in chúng thành các giá trị thứ ba ('.'). Cặp tiếp theo là lũy thừa bậc 1 của sqft và các giá trị dự đoán từ mô hình tuyến tính được vẽ thành một đường '-'.

Không có gì ngạc nhiên khi giá trị dự đoán đều nằm trên một đường, đặc biệt là đường với slope 280 và intercept -43579. Sẽ ra sao nếu chúng ta muốn vẽ một bậc đa thức thứ hai?

In [ ]:
poly2_data = polynomial_dataframe(full_data['sqft_living'], 2) # create polynomial up to 2 degree
features_poly2 = poly2_data.values # extract the features immediately
poly2_data['price'] = full_data['price']
labels_poly2 = poly2_data['price'].values # receive the labels

model2 = LinearRegression().fit(features_poly2, labels_poly2)

In [ ]:
model2.coef_

In [ ]:
plt.plot(poly2_data['power_1'],poly2_data['price'],'.',
        poly2_data['power_1'], model2.predict(features_poly2),'-')

Mô hình kết quả có dạng một nửa parabol. Thử với mô hình với bậc 3 xem nó trông thế nào:

In [ ]:
# 1, 2, 3. Một lần nữa, Ctrl+C, Ctrl+V, chỉnh
poly3_data = polynomial_dataframe(full_data['sqft_living'], 3) # create polynomial up to 2 degree
features_poly3 = poly3_data.values # extract the features immediately
poly3_data['price'] = full_data['price']
labels_poly3 = poly3_data['price'].values # receive the labels

model3 = LinearRegression().fit(features_poly3, labels_poly3)

In [ ]:
# Hãy cẩn thận với thay thế của bạn
model3.coef_

In [ ]:
plt.plot(poly3_data['power_1'],poly3_data['price'],'.',
        poly3_data['power_1'], model3.predict(features_poly3),'-')

Giờ hãy thử đa thức bậc 15:

In [ ]:
# Có rất nhiều cột
poly15_data = polynomial_dataframe(full_data['sqft_living'], 15) # create polynomial up to 2 degree
features_poly15 = poly15_data.values # extract the features immediately
poly15_data['price'] = full_data['price']
labels_poly15 = poly15_data['price'].values # receive the labels

model15 = LinearRegression().fit(features_poly15, labels_poly15)

In [ ]:
# Ctrl+H có thể không ổn nếu bạn không copy từ cell tốt hơn
model15.coef_

In [ ]:
plt.plot(poly15_data['power_1'],poly15_data['price'],'.',
        poly15_data['power_1'], model15.predict(features_poly15),'-')

Đa thức bậc 15 như thế nào? Nó có thích hợp không? Nếu chúng ta thay đổi dữ liệu thì có được đường cong cũng tương tự không? Hãy xem kết quả.

## Thay đổi dữ liệu và học lại

Chúng ta sẽ chia dữ liệu bán hàng thành 4 tập con có kích thước tương tự nhau. Sau đó ước tính một mô hình đa thức bậc 15 cho cả 4 tập con dữ liệu. In ra các hệ số (nên dùng `.coef_` để truy cập các tham số của mô hình) và vẽ biểu đồ kết quả khớp (đã thực hiện phía trên). Quiz sẽ hỏi về các kết quả này.

Để chia dữ liệu bán hàng thành 4 tập con, chúng ta thực hiện như sau:
* Đầu tiên, chia full_data thành 2 tập con với `train_test_split(full_data, train_size=0.5, test_size=0.5 seed=0)`.
* Tiếp theo chia mỗi tập con kết quả thành 2 tập con nữa với `train_test_split(big_set_?, train_size=0.5, test_size=0.5, random_state=0)`.

Chúng ta thiết lập `seed=0` trong các bước này để những người dùng khác nhau sẽ được các kết quả thống nhất.
Chúng ta nên kết thúc với 4 tập con (`set_1`, `set_2`, `set_3`, `set_4`) có kích thước xấp xỉ nhau.

In [ ]:
from sklearn.model_selection import train_test_split
big_set_1, big_set_2 = train_test_split(full_data, train_size=0.5, test_size=0.5, random_state=0)
set_1, set_2 = train_test_split(big_set_1, train_size=0.5, test_size=0.5, random_state=0)
set_3, set_4 = train_test_split(big_set_2, train_size=0.5, test_size=0.5, random_state=0)

**test_size=0.5 có cần thiết không?**

Khớp đa thức bậc 15 vào set_1, set_2, set_3, và set_4 sử dụng sqft_living để đoán giá. Vẽ biểu đồ mô hình kết quả (in các hệ số cho 'power_1' nếu muốn).

In [ ]:
# Bạn có thể vẽ mô hình ở đây
poly15_set1_data = polynomial_dataframe(set_1['sqft_living'], 15) # create polynomial up to 2 degree
features_poly15_set1 = poly15_set1_data.values # extract the features immediately
poly15_set1_data['price'] = set_1['price']
labels_poly15_set1 = poly15_set1_data['price'].values # receive the labels

model15_set1 = LinearRegression().fit(features_poly15_set1, labels_poly15_set1)
print(model15_set1.coef_)
plt.plot(poly15_set1_data['power_1'],poly15_set1_data['price'],'.',
        poly15_set1_data['power_1'], model15_set1.predict(features_poly15_set1),'-')

In [ ]:
# Sau đó copy cho cả 4
# Bạn có thể vẽ mô hình ở đây
poly15_set2_data = polynomial_dataframe(set_2['sqft_living'], 15) # create polynomial up to 2 degree
features_poly15_set2 = poly15_set2_data.values # extract the features immediately
poly15_set2_data['price'] = set_2['price']
labels_poly15_set2 = poly15_set2_data['price'].values # receive the labels

model15_set2 = LinearRegression().fit(features_poly15_set2, labels_poly15_set2)
print(model15_set2.coef_)
plt.plot(poly15_set2_data['power_1'],poly15_set2_data['price'],'.',
        poly15_set2_data['power_1'], model15_set2.predict(features_poly15_set2),'-')

In [ ]:
# Hoặc thực hiện tất cả một lúc
# Bạn có thể vẽ mô hình ở đây
poly15_set3_data = polynomial_dataframe(set_3['sqft_living'], 15) # create polynomial up to 2 degree
features_poly15_set3 = poly15_set3_data.values # extract the features immediately
poly15_set3_data['price'] = set_3['price']
labels_poly15_set3 = poly15_set3_data['price'].values # receive the labels

model15_set3 = LinearRegression().fit(features_poly15_set3, labels_poly15_set3)
print(model15_set3.coef_)
plt.plot(poly15_set3_data['power_1'],poly15_set3_data['price'],'.',
        poly15_set3_data['power_1'], model15_set3.predict(features_poly15_set3),'-')

In [ ]:
# nếu biết lặp, subplot và axes
# Bạn có thể vẽ mô hình ở đây
poly15_set4_data = polynomial_dataframe(set_4['sqft_living'], 15) # create polynomial up to 2 degree
features_poly15_set4 = poly15_set4_data.values # extract the features immediately
poly15_set4_data['price'] = set_4['price']
labels_poly15_set4 = poly15_set4_data['price'].values # receive the labels

model15_set4 = LinearRegression().fit(features_poly15_set4, labels_poly15_set4)
print(model15_set4.coef_)
plt.plot(poly15_set4_data['power_1'],poly15_set4_data['price'],'.',
        poly15_set4_data['power_1'], model15_set4.predict(features_poly15_set4),'-')


**Quiz: Dấu (dương hoặc âm) của power_15 có tương tự ở cả 4 mô hình không??**
Khác nhau

**Quiz: (Đúng/Sai) Đồ thị khớp đã vẽ trông tương tự nhau ở cả 4 mô hình.**
Đúng

# Chọn bậc đa thức

Khi có tham số như bậc của đa thức, có một cách để lựa chọn các tham số này, đó là tập kiểm định. (Chúng ta sẽ khám phá một phương pháp khác sau).

Chúng ta chia tập dữ liệu bán hàng thành tập huấn luyện, tập kiểm tra và tập kiểm đinh như sau:

* Chia dữ liệu bán hàng thành 2 tập: `train_and_validation` và `test_data`. Sử dụng `train_test_split`.
* Tiếp tục chia tập huấn luyện thành 2 tập: `train_data` và `validation_data`. Sử dụng `train_test_split`.

Lần này, chúng ta sẽ thiết lập `random_state=1` để những người dùng có khác nhau có kết quả thống nhất.

In [ ]:
# Không cần import, trừ khi không chạy split phía trên
train_and_validation, test_data = train_test_split(full_data, train_size=0.8, test_size=0.2, random_state=1)
train_data, validation_data = train_test_split(train_and_validation, train_size=0.5, test_size=0.5, random_state=1)

Tiếp theo, chúng ta sẽ viết một vòng lặp như sau:
* Với bậc [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] (để nhận trong python, sử dụng `range(1, 15+1)` tích hợp sẵn)
    * Xây DataFrame của dữ liệu đa thức có train_data['sqft_living'] ở bậc hiện tại <br>
    *Gợi ý:* DataFrame giữ cho `.values` thống nhất với thứ tự cột
    * Load các nhãn của train_data
    * Tìm hiểu mô hình hồi quy đa thức 'sqft' -> 'price' với bậc đó trong dữ liệu HUẤN LUYỆN
    * Tính RSS trong dữ liệu KIỂM ĐỊNH (ở đây sẽ sử dụng `.predict()`) cho các bậc đó. Chúng ta sẽ cần áp dụng tất cả các bước trên vào tập kiểm định.
* Báo lại bậc nào có RSS thấp nhất trong dữ liệu kiểm định (chỉ mục python từ 0).

In [ ]:
# Đọc hướng dẫn và tiến hành. Chúng ta có thể loại một số giá trị ra khỏi vòng lặp
RSS_list = []
for degree in range(1, 16):
    poly_data = polynomial_dataframe(train_data['sqft_living'], degree)
    feature_data = poly_data.values
    poly_data['price'] = train_data['price']
    label_data = poly_data['price'].values
    model = LinearRegression().fit(feature_data, label_data)
    poly_valid_data = polynomial_dataframe(validation_data['sqft_living'], degree)
    feature_valid_data = poly_valid_data.values
    label_valid_data = validation_data['price'].values
    predictions = model.predict(feature_valid_data)
    residual = predictions - label_valid_data
    RSS = np.sum(residual**2)
    RSS_list.append(RSS)
    
min_RSS = min(RSS_list)
degree_min = RSS_list.index(min_RSS)
print(degree_min+1)

**Quiz: Bậc nào (1, 2, …, 15) có RSS thấp nhất trong dữ liệu KIỂM ĐỊNH?**
Bậc 5 có RSS thấp nhất trong tập KIỂM ĐỊNH

Chúng ta đã chọn bậc của đa thức sử dụng dữ liệu kiểm định, hãy tính RSS của mô hình này trong dữ liệu KIỂM TRA. Báo cáo RSS trong quiz.

In [ ]:
# Mức độ trợ giúp: thấp
poly_data = polynomial_dataframe(train_data['sqft_living'], 6)
feature_data = poly_data.values
poly_data['price'] = train_data['price']
label_data = poly_data['price']
model = LinearRegression().fit(feature_data, label_data)
poly_test_data = polynomial_dataframe(test_data['sqft_living'], 6)
feature_test_data = poly_test_data.values
label_test_data = test_data['price']
predictions = model.predict(feature_test_data)
residual = predictions - label_test_data
RSS = np.sum(residual*residual)
print(RSS)

**Quiz: RSS trong dữ liệu KIỂM TRA cho mô hình với bậc được chọn từ dữ liệu KIỂM ĐỊNH là?**
266245573158238.0